In [3]:
import numpy as np
import os
import time
import efficientnet.keras as efn 
import efficientnet.tfkeras
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold
from keras.layers import Dropout
from keras.layers.pooling import GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
import json
from keras.models import model_from_json, load_model
from keras.preprocessing.image import load_img
import argparse
import pickle
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from PIL import Image
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [4]:
!unzip gdrive/My\ Drive/imagesdata1.zip> /dev/null

In [5]:
#Map each class to an integer label
class_mapping = {}
class_images1 = {}

In [6]:
os.listdir('/content/imagesdata1/data')

['2000 Normal Skin',
 '2000 Blepharitis',
 '2000 Eczema',
 '2000 Angioedema',
 '2000 No-Face',
 '2000 Melasma',
 '2000 Acne',
 '2000 Rosacea',
 '2000 Vitiligo',
 '2000 Actinic keratosis']

In [7]:
# Create dictionary to map integer labels to individuals
# Class_images will record number of images for each class
for index, directory in enumerate(os.listdir('/content/imagesdata1/data')):
    class_mapping[index] = directory.split(" ")[1]
    class_images1[index] = int(directory.split(' ')[0])

In [8]:
class_mapping

{0: 'Normal',
 1: 'Blepharitis',
 2: 'Eczema',
 3: 'Angioedema',
 4: 'No-Face',
 5: 'Melasma',
 6: 'Acne',
 7: 'Rosacea',
 8: 'Vitiligo',
 9: 'Actinic'}

In [9]:
class_images1

{0: 2000,
 1: 2000,
 2: 2000,
 3: 2000,
 4: 2000,
 5: 2000,
 6: 2000,
 7: 2000,
 8: 2000,
 9: 2000}

In [10]:
#We can look at the distribution of each class.
total_num_images = np.sum(list(class_images1.values()))

print("Skin diseases Training \t Composition of Dataset\n")
for label, num_images in class_images1.items():
    print("{:20} {:.2f}%".format(
        class_mapping[label], (num_images / total_num_images) * 100))

Skin diseases Training 	 Composition of Dataset

Normal               10.00%
Blepharitis          10.00%
Eczema               10.00%
Angioedema           10.00%
No-Face              10.00%
Melasma              10.00%
Acne                 10.00%
Rosacea              10.00%
Vitiligo             10.00%
Actinic              10.00%


In [11]:
#Read the images into RGB (red-green-blue) pixel value arrays.
import matplotlib.pyplot as plt
#from scipy.misc import imresize
image_arrays = []
image_labels = []
root_image_directory = "/content/imagesdata1/data/"
target_height=224
target_width=224
classes=np.zeros((20000,2))
counter=0
for label, person in class_mapping.items():

    for directory in os.listdir(root_image_directory):
        if directory.split(" ")[1] == person:
            image_directory = root_image_directory + directory
           
            break
               
    for image in os.listdir(image_directory):
       
        sub_id=image[:image.index('-')]
        image = plt.imread(os.path.join(image_directory, image))
       #image = imresize(image, (target_width, target_height))
        image =np.array(Image.fromarray(image).resize((target_width,target_height),resample=Image.BICUBIC))
        image_arrays.append(image)
        image_labels.append(label)
        classes[counter,0]=label
        classes[counter,1]=sub_id
        counter=counter+1



In [12]:
image_arrays = np.array(image_arrays)
image_labels = np.array(image_labels)
print(image_arrays.shape, image_labels.shape)


(20000, 224, 224, 3) (20000,)


In [13]:
num_classes = 10

# convert class labels to on-hot encoding
Y = np_utils.to_categorical(image_labels, num_classes)
print('Example After conversion to one-hot:', Y[0])


Example After conversion to one-hot: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [14]:
#Shuffle the dataset
x,y = shuffle(image_arrays,Y, random_state=2)

In [15]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(14000, 224, 224, 3) (6000, 224, 224, 3)
(14000, 10) (6000, 10)


In [16]:
import tensorflow as tf
import keras


In [17]:
# Custom_vgg_model_1
#Training the classifier alone
image_input = Input(shape=(224, 224, 3))
model1 = tf.keras.applications.EfficientNetB0(weights='imagenet',include_top=False,input_tensor=image_input)
#model1 = VGG16(weights='imagenet',include_top=False,input_tensor=image_input)
model1.summary()



for i, layer in enumerate(model1.layers):
    print(i, layer.name, layer.trainable)

from keras.layers import Dropout
from keras.layers.pooling import GlobalAveragePooling2D
x = model1.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=model1.input, outputs=predictions)
print(model.summary())


16719872/16705208 [==============================] - 0s 0us/step
Model: "efficientnetb0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 224, 224, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization[0][0]              
____________________

In [18]:
# Make the conv_base NOT trainable:

for layer in model1.layers[:]:
    layer.trainable = False

print('conv_base is now NOT trainable')


conv_base is now NOT trainable


In [21]:
for layer in model1.layers[:162]:
    layer.trainable = False
for layer in model1.layers[162:]:
    layer.trainable = True

print('block 6&7 of the conv_base is now trainable')

block 6&7 of the conv_base is now trainable


In [22]:
for i, layer in enumerate(model1.layers):
    print(i, layer.name, layer.trainable)

0 input_1 False
1 rescaling False
2 normalization False
3 stem_conv_pad False
4 stem_conv False
5 stem_bn False
6 stem_activation False
7 block1a_dwconv False
8 block1a_bn False
9 block1a_activation False
10 block1a_se_squeeze False
11 block1a_se_reshape False
12 block1a_se_reduce False
13 block1a_se_expand False
14 block1a_se_excite False
15 block1a_project_conv False
16 block1a_project_bn False
17 block2a_expand_conv False
18 block2a_expand_bn False
19 block2a_expand_activation False
20 block2a_dwconv_pad False
21 block2a_dwconv False
22 block2a_bn False
23 block2a_activation False
24 block2a_se_squeeze False
25 block2a_se_reshape False
26 block2a_se_reduce False
27 block2a_se_expand False
28 block2a_se_excite False
29 block2a_project_conv False
30 block2a_project_bn False
31 block2b_expand_conv False
32 block2b_expand_bn False
33 block2b_expand_activation False
34 block2b_dwconv False
35 block2b_bn False
36 block2b_activation False
37 block2b_se_squeeze False
38 block2b_se_reshape F

In [23]:

#opt = Adam(lr=1e-4)
#model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=["accuracy"])
opt = SGD(lr=1e-4, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 224, 224, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization[0][0]              
______________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
t=time.time()
hist = model.fit(X_train, y_train, batch_size=16, epochs=50, verbose=1, validation_data=(X_test, y_test))
(loss, accuracy) = model.evaluate(X_test, y_test, batch_size=10, verbose=1)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

Epoch 1/50
875/875 [==============================] - 87s 59ms/step - loss: 2.0273 - accuracy: 0.2904 - val_loss: 1.6042 - val_accuracy: 0.5543
Epoch 2/50
875/875 [==============================] - 60s 68ms/step - loss: 1.5263 - accuracy: 0.5281 - val_loss: 1.2050 - val_accuracy: 0.6900
Epoch 3/50
875/875 [==============================] - 60s 69ms/step - loss: 1.2147 - accuracy: 0.6293 - val_loss: 0.9481 - val_accuracy: 0.7537
Epoch 4/50
875/875 [==============================] - 60s 69ms/step - loss: 1.0189 - accuracy: 0.6859 - val_loss: 0.7746 - val_accuracy: 0.7970
Epoch 5/50
875/875 [==============================] - 60s 69ms/step - loss: 0.8824 - accuracy: 0.7252 - val_loss: 0.6549 - val_accuracy: 0.8240
Epoch 6/50
875/875 [==============================] - 50s 58ms/step - loss: 0.7829 - accuracy: 0.7542 - val_loss: 0.5688 - val_accuracy: 0.8450
Epoch 7/50
875/875 [==============================] - 60s 69ms/step - loss: 0.7110 - accuracy: 0.7751 - val_loss: 0.5065 - val_accuracy:

In [1]:
tf.keras.utils.plot_model(model,show_shapes=True)

NameError: ignored

In [ ]:
#tf.keras.models.save_model(my_model, './saved_models/my_tf_model')
model.save('./saved_models/my_tf_model')

In [ ]:
import matplotlib.pyplot as plt
# visualizing losses and accuracy
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['accuracy']
val_acc=hist.history['val_accuracy']
xc=range(50)
epochs = range(len(train_acc))

In [ ]:
plt.plot(epochs, train_acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs,train_loss , 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import json
from keras.models import model_from_json, load_model
import numpy
import os  

# serialize model to JSON
model_json = model.to_json()
with open("savemodel_Shuffeling-4Blocks - 50 Epochs - SGD_efficientNet.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("savemodel_Shuffeling-4Blocks - 50 Epochs - SGD_efficientNet.h5")
print("Saved model to disk")

In [ ]:
# load json and create model
json_file = open('savemodel_Shuffeling-4Blocks - 50 Epochs - SGD_efficientNet.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("savemodel_Shuffeling-4Blocks - 50 Epochs - SGD_efficientNet.h5")
print("Loaded model from disk")

In [ ]:
from sklearn.metrics import confusion_matrix
import keras
preds=loaded_model.predict(np.array(X_test))
predicted_classes = np.argmax(np.round(preds),axis=1)
predicted_classes1 = np.array(predicted_classes)
y_pred=predicted_classes1
testy = np.argmax(np.round(y_test),axis=1)
ytrue = np.array(testy)
print(y_pred)
print(ytrue)

In [ ]:
classes=['Acne','Actinic','Angio','Bleph','Ecz','Melas','No-Face','Norm','Rosa','Viti']

def plot_confusion_matrix(ytrue, y_pred,classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(ytrue, y_pred)
    # Only use the labels that appear in the data
    
    classes=['Acne','Actinic','Angio','Bleph','Ecz','Melas','No-Face','Norm','Rosa','Viti']
   
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax
np.set_printoptions(precision=2)
# Plot non-normalized confusion matrix
plot_confusion_matrix(ytrue, y_pred, classes=classes,
                      title='Confusion matrix, without normalization in case 70-30')

# Plot normalized confusion matrix
plot_confusion_matrix(ytrue, y_pred, classes=classes, normalize=True,
                      title='Normalized confusion matrix in case 70-30')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import classification_report
y_actu = pd.Series(ytrue, name='Actual')
y_pred = pd.Series(y_pred, name='Predicted')
target_names=['Acne','Actinic','Angio','Bleph','Ecz','Melas','No-Face','Norm','Rosa','Viti']
print(classification_report(y_actu, y_pred, target_names=target_names))